# Ejemplo de simulación discreta: Filas de espera en Carrefour
---
- Autor: **[Rodrigo Maranzana](https://www.linkedin.com/in/rodrigo-maranzana/)**
- Año: **2020**
- Licencia: **[MIT License](https://github.com/harb-OR/casos_practicos/blob/master/LICENSE)**
---

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Resolución" data-toc-modified-id="Resolución-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Resolución</a></span><ul class="toc-item"><li><span><a href="#Simulador-de-eventos-discretos:" data-toc-modified-id="Simulador-de-eventos-discretos:-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Simulador de eventos discretos:</a></span><ul class="toc-item"><li><span><a href="#Función-de-resultados-del-simulador:" data-toc-modified-id="Función-de-resultados-del-simulador:-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Función de resultados del simulador:</a></span></li><li><span><a href="#Estructura-del-simulador:" data-toc-modified-id="Estructura-del-simulador:-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Estructura del simulador:</a></span></li></ul></li><li><span><a href="#Método-de-Monte-Carlo-aplicado-a-filas-de-espera:" data-toc-modified-id="Método-de-Monte-Carlo-aplicado-a-filas-de-espera:-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Método de Monte Carlo aplicado a filas de espera:</a></span></li></ul></li></ul></div>

Se busca estimar las métricas para un modelo de filas de espera con distribuciones de llegadas y salidas que tienen parámetros dinámicos en cada tiempo simulado. Esto es consecuencia de un requerimiento de modelado que proviene del análisis cualitativo de psicología de filas.

Se busca comparar un modelo M/M/1 con Mvar/Mvar/1, en donde Mvar indica la distribución exponencial de parámetro variable.
Los parámetros son función del largo de las filas. Concretamente se eligió arbitrariamente una función sigmoidal.

## Resolución

En primer lugar, importamos las librerías necesarias para el cálculo:

In [334]:
import numpy as np
from numpy.random import exponential
import matplotlib.pyplot as plt
import pandas as pd
import heapq
from time import time
from scipy.stats import logistic

### Simulador de eventos discretos:

Definimos funciones asociadas a los eventos:

In [335]:
# Funciones pertenecientes a eventos:
def generar_llegada(t_global, generador_codigo_persona, lambd):
    t_llegada = exponential(1/lambd) + t_global
    nueva_n_persona = next(generador_codigo_persona)
    return t_llegada, nueva_n_persona

def generar_salida_servidor(t_global, mu):
    t_salida = exponential(1/mu) + t_global
    return t_salida

#### Función de resultados del simulador:

In [336]:
## Función para caclular el tiempo medio que un producto pasa en la fila:
def calcular_q_media(estado_fila_array, t_array):
    tiempo_total = t_array[-1]
    t_entre_eventos = np.diff(np.insert(t_array, 0, 0)).transpose()
    
    return np.dot(estado_fila_array, t_entre_eventos) / tiempo_total

def calculo_metricas(tabla_eventos, t_array, estado_fila, estado_sistema):
    # Cálculo de Wq y Ws:
    ws_acum = 0
    wq_acum = 0

    for key, value in tabla_eventos.items():
        # los ws y wq los vamos a calcular con las diferencias entre el tiempo en sistema y el tiempo en fila de c/persona.
        if 'salida_servidor' in value:
            ws_acum += value['salida_servidor'] - value['nueva_persona']

        if 'salida_fila' in value:
            wq_acum += value['salida_fila'] - value['nueva_persona']

    wq = wq_acum / (key + 1)
    ws = ws_acum / (key + 1)

    # Cálculo de Lq y Ls:
    ## Calculamos lq y ls:
    lq = calcular_q_media(estado_fila, t_array)
    ls = calcular_q_media(estado_sistema, t_array)

    # Cantidad de personas que salieron del sistema:
    q_personas = sorted(tabla_eventos.keys())[-1]
    
    return wq, ws, lq, ls, q_personas

#### Estructura del simulador:
Escribimos la estructura del simulador a través de un while loop. Esta vez el simulador será una función ya que lo usaremos bajo el método de Monte Carlo.

In [337]:
def simulador(t_corte, n_servidores, lambd, mu):
    
    # Inicialización de fila simulador:
    fila_simulador = []
    def ingresar_a_fila_simulador(t, tipo_evento, n_persona, n_servidor):
        heapq.heappush(fila_simulador, (t, tipo_evento, n_persona, n_servidor))
    
    def recuperar_de_fila_simulador():
        return heapq.heappop(fila_simulador)
    
    # Inicialización de fila caso:
    fila = []
    def entrada_fila(n_persona):
        heapq.heappush(fila, n_persona)
    
    def salida_fila():
        return heapq.heappop(fila)
    
    # Lambda y Mu variables:
    def calculate_mu(mu, length_fila):
#         return mu*(1 - logistic.cdf(length_fila, 9, 4))
        return mu

    def calculate_lambd(lambd, length_fila):
#         return lambd*(1 - logistic.cdf(length_fila, 9, 4))
        return lambd

    # Inicialización de variables:
    t_array = []
    tabla_eventos = {}
    estado_fila = []
    estado_sistema = []
    gen_codigo_personas = iter(range(0, 20000))
    t_global = 0 # Tiempo actual de simulación.
    n_persona = 0 # Número de producto en el tiempo actual de simulación.
    estado_servidores = np.array([True]*n_servidores) # Estado libre de cada máquina: True (libre), False (en uso)
    
    ################## SIMULADOR #############################
    # Inicio
    t_llegada, n_nueva_persona = generar_llegada(t_global, gen_codigo_personas, calculate_lambd(lambd, len(fila)))
    ingresar_a_fila_simulador(t_llegada, 'nueva_persona', n_nueva_persona, None)
    
    while True:
        # Sacar evento de la fila de eventos:
        nuevo_evento = recuperar_de_fila_simulador()
        t_global = nuevo_evento[0]
        if t_global > t_corte:
            break
        tipo_evento = nuevo_evento[1]
        n_persona = nuevo_evento[2]
        n_servidor = nuevo_evento[3]
        
        # Agregar eventos a la tabla de seguimiento:
        evento_a_tabla = {tipo_evento: t_global}
        if n_persona in tabla_eventos:
            tabla_eventos[n_persona].update(evento_a_tabla)
        else:        
            tabla_eventos.update({n_persona: evento_a_tabla})

       ################ EVENTO: llegada de una persona ################
        if tipo_evento == 'nueva_persona':
            servidores_libres = np.argwhere(estado_servidores)
            if servidores_libres.size != 0:
                # Se elige el primer servidor libre:
                index_servidor = servidores_libres[0][0]
                estado_servidores[index_servidor] = False
                t_salida = generar_salida_servidor(t_global, calculate_mu(mu, len(fila)))
                ingresar_a_fila_simulador(t_salida, 'salida_servidor', n_persona, index_servidor)
            else:
                entrada_fila(n_persona)

            t_llegada, n_nueva_persona = generar_llegada(t_global, gen_codigo_personas, calculate_lambd(lambd, len(fila)))
            ingresar_a_fila_simulador(t_llegada, 'nueva_persona', n_nueva_persona, None)

        ################ EVENTO: salida de servidor ################
        if tipo_evento == 'salida_servidor':
            # ingresar nueva persona:
            if len(fila) > 0:
                n_persona = salida_fila()
                t_salida = generar_salida_servidor(t_global, calculate_mu(mu, len(fila)))
                ingresar_a_fila_simulador(t_salida, 'salida_servidor', n_persona, n_servidor)

                # Agregamos un evento virtual para registrar la salida de la fila.
                # Este evento no forma parte de los eventos del simulador.  
                evento_a_tabla = {'salida_fila': t_global}
                tabla_eventos[n_persona].update(evento_a_tabla)
            else:
                estado_servidores[n_servidor] = True

        ####### Recolectar datos ##########
        t_array.append(t_global)
        
        q_fila = len(fila)
        estado_fila.append(q_fila)
        
        q_sistema = q_fila + np.sum(~estado_servidores)
        estado_sistema.append(q_sistema)
        
    ###### Cálculo de resultados ##########
    return calculo_metricas(tabla_eventos, t_array, estado_fila, estado_sistema)

### Método de Monte Carlo aplicado a filas de espera:

Iteramos n veces y luego sacamos la media de las métricas Wq, Ws, Lq y Ls.

In [338]:
# Parámetros:
iteraciones = 10

n_servidores = 1
t_corte = 240
lambd = n_servidores*9
mu = 12

# Inicialización:
wq_array = []
ws_array = []
lq_array = []
ls_array = []
q_personas_array = []

t_inicio = time() # guardamos tiempo inicial 

# Iteramos:
for i in range(0, iteraciones):
    # Ejecutar simulador:
    wq_i, ws_i, lq_i, ls_i, q_personas_i = simulador(t_corte, n_servidores, lambd, mu)
    
    # Agregar resultados a array:
    wq_array.append(wq_i)
    ws_array.append(ws_i)
    lq_array.append(lq_i)
    ls_array.append(ls_i)
    q_personas_array.append(q_personas_i)

t_fin = time() # guardamos tiempo final
    
wq = np.mean(wq_array)
ws = np.mean(ws_array)
lq = np.mean(lq_array)
ls = np.mean(ls_array)
q_personas = np.mean(q_personas_array)

# Resultados:
print('######## Resultados ##############')
print('## Monte Carlo ##')
print('Iteraciones de Monte Carlo: %i' % iteraciones, 
      '\nTiempo de cálculo: %0.2f' % (t_fin - t_inicio))
print('\n## Filas de espera ##')
print('Cantidad de servidores: %s' % n_servidores, 
      '\nTiempo medio en fila: %0.2f' % wq, 
      '\nTiempo medio en sistema: %0.2f' % ws, 
      '\nLargo medio de la fila: %0.2f' % lq, 
      '\nCantidad media de personas en sistema: %0.2f' % ls, 
      '\nPersonas salidas del sistema en simulación: %i' % q_personas)

######## Resultados ##############
## Monte Carlo ##
Iteraciones de Monte Carlo: 10 
Tiempo de cálculo: 1.02

## Filas de espera ##
Cantidad de servidores: 1 
Tiempo medio en fila: 0.26 
Tiempo medio en sistema: 0.35 
Largo medio de la fila: 2.39 
Cantidad media de personas en sistema: 3.28 
Personas salidas del sistema en simulación: 2165
